<a href="https://colab.research.google.com/github/MostHumble/EN2KAB/blob/main/train_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install tokenizers transformers

In [ ]:
from google.colab import files
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from transformers import PreTrainedTokenizerFast
import os
import pandas as pd


In [ ]:
# Choose the Kaggle API token JSON file that you downloaded
files.upload()
%%capture
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sifalklioui/wiki-kabyle
!unzip wiki-kabyle.zip -d data

In [ ]:
# For the user's convenience `tokenizers` provides some very high-level classes encapsulating
# the overall pipeline for various well-known tokenization algorithm.
# Everything described below can be replaced by the ByteLevelBPETokenizer class.



special_tokens = {'unk_token':"[UNK]",'cls_token ':"[CLS]",'sep_token':"[SEP]", 'pad_token':"[PAD]",'mask_token':"[MASK]",'bos_token':"[BOS]"}
path = "/content/data"
files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".txt")]

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

tokenizer.pre_tokenizer = Whitespace()


# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=35000, show_progress=True,special_tokens=list(special_tokens.values()))
tokenizer.train(files, trainer)
print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

In [10]:
# Load trained tokenizer for future use
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer,**special_tokens)
# Loading the data to try the tokinzer
df = pd.read_csv('/content/drive/MyDrive/eng2kab.tsv',sep='\t',names=['id1','en','id2','kab'], header=None).drop(columns=['id1','id2'])

In [14]:
fast_tokenizer(df.kab[0],max_length=6,padding='max_length')

{'input_ids': [1462, 3101, 6, 3, 3, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 0, 0, 0]}